In [ ]:
import requests
import json
import math

import numpy as np
import pandas as pd


In [ ]:
# get hashtag ids from a list of hashtags - hashtag id is used to retrive videos for the next step

hashtag_names = ['Interested Hashtag'] # You can customize your own hashtag list

get_hashid_url = "https://tiktok-scraper2.p.rapidapi.com/hashtag/info"
headers = {
	"X-RapidAPI-Key": "YourOwnKey",
	"X-RapidAPI-Host": "YourOwnHost"
}

hashtag_info_l = []
for name in hashtag_names: 
    querystring = {"hashtag":name}
    hashid_response = requests.request("GET", get_hashid_url, headers=headers, params=querystring)
    
    hashtag_json = json.loads(hashid_response.text)
    hashid = hashtag_json['challengeInfo']['challenge']['id']
    video_count = hashtag_json['challengeInfo']['stats']['videoCount']
    
    print('hashtag: '+ name + "\tid: " + hashid + '\tvideos count: ' + str(video_count))
    hashtag_info_l.append((name,hashid,video_count))

In [ ]:
# a method to get videos from one hashtag

def getVideos(name, hashid, video_count):
    
    get_videos_url = "https://tiktok-scraper2.p.rapidapi.com/hashtag/videos"
    
    headers = {
        "X-RapidAPI-Key": "f823b922edmshcab34c928964e2dp1b4ce1jsnc0b4e45b7e3b",
        "X-RapidAPI-Host": "tiktok-scraper2.p.rapidapi.com"
    }
    
    # use video_count to identify pages 
    # There might be a bug because the api is not able to retrive all the videos from each hashtag
    pages = math.ceil(video_count / 30)
    
    # create lists to store videos' information
    video_id_l = []
    video_desc_l = []
    video_likes_count_l = []
    video_comments_count_l = []
    video_timestamp_l = []
    video_url_l = []
    video_hashtags_l = []
    
    # page loop
    for i in range(pages):
        querystring = {"hashtag_id":hashid,"cursor":i*30}
        response_videos = requests.request("GET", get_videos_url, headers=headers, params=querystring)
        json_videos = json.loads(response_videos.text)
        
        try:
            video_items = json_videos['itemList']
        
            # on each page, get information of each video and save it to the lists created above
            for video in video_items:
                hashtags_l = []
                
                for hashtag_item in video['challenges']:
                    hashtags_l.append(hashtag_item['title'])
                
                # get information of each video (id, description, time, url,etc)
                video_id = video['id']
                video_desc = video['desc']
                video_likes_count = video['stats']['diggCount']
                video_comments_count = video['stats']['commentCount']
                video_timestamp = video['createTime']
                video_author = video['author']['uniqueId']
                video_url = 'https://www.tiktok.com/@'+ video_author +'/video/'+ video_id # create video url using author name and video id
                video_hashtags = ' '.join(hashtags_l)
                
                # add video information to each list
                video_id_l.append(video_id)
                video_desc_l.append(video_desc)
                video_likes_count_l.append(video_likes_count)
                video_comments_count_l.append(video_comments_count)
                video_timestamp_l.append(video_timestamp)
                video_url_l.append(video_url)
                video_hashtags_l.append(video_hashtags)
                    
        except:
            pass
            
        if json_videos == {}:
            break
            
        if json_videos['hasMore'] == False:
            break
            
    # save all videos information to a dictionary
    video_dic = {'video_id':video_id_l,
                 'video_desc':video_desc_l,
                 'likes_count': video_likes_count_l,
                 'comments_count': video_comments_count_l,
                 'video_timestamp':video_timestamp_l,
                 'video_url':video_url_l,
                 'video_hashtags':video_hashtags_l}
    
    # create a data frame for videos
    video_df = pd.DataFrame(video_dic)
    
    return video_df
    
    

In [ ]:
# Some videos may be under more than one hashtag
# Create a method to delate the videos included in the previous hashtags

def deleteDuplicates(df1, df2):
    
    # create a list of the video ids in the previous data frame
    id1_l = list(df1['video_id'])
    
    # create a list of duplicated indexes
    del_index_l = []
    
    for i,id2 in enumerate(df2['video_id']):
        if str(id2) in id1_l:
            del_index_l.append(i)
    
    # drop duplicated rows
    df2 = df2.drop(index=del_index_l)
    
    return df2

In [ ]:
# get all videos from hashtag list


for i,(name,hashid,video_count) in enumerate(hashtag_info_l):
    
    if i == 0:
        video_df = getVideos(name,hashid,video_count)
        # change long numbers to string format so the value will not be changed in the excel files
        video_df = video_df.astype({"video_id": str,'video_timestamp':str})
        video_df.to_excel("data/videos/videos_{}.xlsx".format(name), sheet_name='videos', index=False)
        
    else:
        df_sub = getVideos(name,hashid,video_count)
        df_sub = df_sub.astype({"video_id": str,'video_timestamp':str})
        df_sub = deleteDuplicates(video_df, df_sub)
        
        # save videos dataframe seprately for getting comments
        df_sub.to_excel("data/videos/videos_{}.xlsx".format(name), sheet_name='videos', index=False)
        
        # concat dfs 
        video_df = pd.concat([video_df,df_sub], ignore_index=True)
        
# save dataframe of all videos to excel
video_df.to_excel("data/videos_all.xlsx",sheet_name='videos', index=False) 

video_df
